In [1]:
import pickle

import altair as alt
import numpy as np
import pandas as pd
import re
import spacy


from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamulticore import LdaMulticore

In [2]:
pd.options.display.max_colwidth = 200
DEFAULT_WIDTH = 800

In [3]:
with open("canebiere.pickle", "rb") as src:
    df: pd.DataFrame = pickle.load(src)

# La Canebière Académie, des origines à aujourd'hui

## Publications par auteur

In [4]:
alt.Chart(df[["Auteur", "Publications"]]).mark_bar().encode(
    x=alt.X(
        "Auteur:N",
        axis=alt.Axis(labelAngle=0)
    ),
    y=alt.Y(
        "count(Publications):Q",
        axis=alt.Axis(title="Académies")
    ),
    color="Auteur:N",
    tooltip=["Auteur", alt.Tooltip("count(Publications):Q", title="Académies")]
).properties(
    width=DEFAULT_WIDTH
)

alt.Chart(...)

In [5]:
alt.Chart(df[["Auteur", "Publications"]]).mark_arc(innerRadius=50, outerRadius=300).encode(
    theta=alt.Theta(
        "count(Publications):Q",
        title="Académies"
    ),
    color="Auteur:N",
    tooltip=["Auteur", alt.Tooltip("count(Publications):Q", title="Académies")]
).properties(
    width=DEFAULT_WIDTH
)

alt.Chart(...)

## Publications par Année et Auteur

In [7]:
raw_counts = alt.Chart(df[["Date", "Auteur", "Publications"]]).mark_bar().encode(
    x=alt.X("year(Date):O", title=None, axis=alt.Axis(ticks=False, labels=False)),
    y=alt.Y("count():Q", title="Académies"),
    color="Auteur:N",
    tooltip=[alt.Tooltip("year(Date)", title="Année"), "Auteur", alt.Tooltip("count()", title="Académies")]
).properties(
    width=DEFAULT_WIDTH
)

pct_counts = alt.Chart(df[["Date", "Auteur", "Publications"]]).mark_bar().encode(
    x=alt.X("year(Date):O", title="Année", axis=alt.Axis(ticks=False)),
    y=alt.Y("count():Q", title="Académies", stack="normalize", axis=alt.Axis(format="%")),
    color="Auteur:N",
    tooltip=[alt.Tooltip("year(Date)", title="Année"), "Auteur", alt.Tooltip("count()", title="Académies")]
).properties(
    width=DEFAULT_WIDTH
)

raw_counts & pct_counts

alt.VConcatChart(...)

## Longueur du texte par auteur

In [8]:
alt.Chart(df[["Auteur", "nb_tokens"]]).transform_density(
    'nb_tokens',
    as_=['Nombre de Mots', 'density'],
    extent=[0, 6000],
    groupby=['Auteur']
).mark_area(orient='horizontal').encode(
    y='Nombre de Mots:Q',
    color='Auteur:N',
    x=alt.X(
        'density:Q',
        stack='center',
        impute=None,
        title=None,
        axis=alt.Axis(labels=False, values=[0],grid=False, ticks=True),
    ),
    column=alt.Column(
        'Auteur:N',
        header=alt.Header(
            titleOrient='bottom',
            labelOrient='bottom',
            labelPadding=0,
            labelFontWeight="bold",
            labelBaseline="bottom"
        ),
    )
).properties(
    width=200
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
)

alt.Chart(...)



___


# Blaah l'académicien

* 331 académies, 48512 phrases, 850787 mots

In [9]:
df = df.set_index("Auteur")
blaah = df.loc["Blaah"].copy().reset_index(drop=True)

## Mots les plus utilisés

In [10]:
lemmatizer = FrenchLefffLemmatizer()

def lemmatize(x: spacy.tokens.Token) -> str:
    pos_2_leff = {
        "VERB": "v",
        "NOUN": "n",
        "ADJ": "a"
    }
    leff_pos = pos_2_leff.get(x.pos_, "n")
    return lemmatizer.lemmatize(word=x.lemma_, pos=leff_pos)

corpus = [[lemmatize(t) for t in doc if not t.is_stop and not t.is_space and len(t) > 2] for doc in blaah["docs"]]
dic = Dictionary(documents=corpus)

## Mots utilisés dans le plus d'académies

In [11]:
dfs = pd.DataFrame([{"word": dic[k], "dfs": v} for k, v in dic.dfs.items()])

alt.Chart(dfs[:100]).mark_bar().encode(
    y=alt.Y("dfs:Q", title="Nombre d'académies utilisant le mot"),
    x=alt.X(
        "word:O", 
        sort=alt.EncodingSortField(field="dfs", order="descending"),
        axis=alt.Axis(labelAngle=-45),
        title="Mot"
    ),
    color=alt.Color("dfs", scale=alt.Scale(scheme="redyellowgreen")),
    tooltip=[alt.Tooltip(field="word", title="Mot"), alt.Tooltip(field="dfs", title="# Académies")]
).properties(width=1200)

alt.Chart(...)

## Mots les plus fréquents

In [12]:
cfs = pd.DataFrame([{"word": dic[k], "cfs": v} for k, v in dic.cfs.items()])

alt.Chart(cfs[:100]).mark_bar().encode(
    y=alt.Y("cfs:Q", title="Nombre d'utilisations du mot"),
    x=alt.X(
        "word:O", 
        sort=alt.EncodingSortField(field="cfs", order="descending"),
        axis=alt.Axis(labelAngle=-45),
        title="Mot"
    ),
    color=alt.Color("cfs", scale=alt.Scale(scheme="redyellowgreen")),
    tooltip=[alt.Tooltip(field="word", title="Mot"), alt.Tooltip(field="cfs", title="# Utilisations")]
).properties(width=1200)

alt.Chart(...)

In [13]:
# Les Titres de Blaah

In [14]:
# * Le modèle: "xxx-OM (score), La Canebière académie VERBE COD..."
# * Lyon-OM (2-1), La Canebière académie ne tient pas la distance

In [15]:
home = re.compile(r"^\s*OM\s*[\-–]\s*(?P<adversaire>[\w\- ]+)")
away = re.compile(r"^\s*(?P<adversaire>[\w\- ]+)\s*[\-–]\s*OM")

In [16]:
# ## Les exceptions à la règle

# * Les académies de rétrospective
# * On les sort du corpus
# * Reste 318 académies

In [17]:
test = [(home.match(txt) is not None) is not (away.match(txt) is not None) for txt in blaah["Titre"]]

In [18]:
blaah = blaah.iloc[[x[0] for x in filter(lambda t: t[1], enumerate(test))]].copy()

In [19]:
def extract_adversaire(x):
    m = home.match(x)
    if m is None:
        m = away.match(x)

    if m is None:
        raise ValueError(f"Can't process {x}")
    return m.group("adversaire").strip()

blaah["Domicile"] = blaah["Titre"].apply(lambda x: home.match(x) is not None)
blaah["Adversaire"] = blaah["Titre"].apply(extract_adversaire)



___


# Scores (Extraits du Titre)

In [20]:
score = re.compile(r"^\s*\((?P<receveuse>\d+)\s*-\s*(?P<visiteuse>\d+).*\)")

In [21]:
def apres_equipes(x):
    m = home.match(x)
    if m is None:
        m = away.match(x)
    if m is None:
        raise ValueError(f"Can't process {x}")

    return x[m.end():]

In [22]:
def extract_buts(x):
    txt = apres_equipes(x)
    m = score.match(txt)
    if m is None:
        return pd.Series([np.nan, np.nan], index=["Buts_Receveuse", "Buts_Visiteuse"])
    return pd.Series([int(m.group("receveuse")), int(m.group("visiteuse"))], index=["Buts_Receveuse", "Buts_Visiteuse"])

In [23]:
buts = blaah["Titre"].apply(extract_buts)
blaah[["Buts_R", "Buts_V"]] = buts
blaah["Buts_OM"] = blaah.apply(lambda x: x["Buts_R"] if x["Domicile"] else x["Buts_V"], axis=1)
blaah["Buts_Adversaire"] = blaah.apply(lambda x: x["Buts_V"] if x["Domicile"] else x["Buts_R"], axis=1)
blaah.drop(columns=["Buts_R", "Buts_V"], inplace=True)
blaah.dropna(subset=["Buts_OM", "Buts_Adversaire"], inplace=True)
for col in ["Buts_OM", "Buts_Adversaire"]:
    blaah[col] = blaah[col].astype(int)

In [24]:
def resultat(x):
    om = x["Buts_OM"]
    adv = x["Buts_Adversaire"]

    if om > adv:
        return "Victoire"
    if om == adv:
        return "Nul"
    return "Défaite"

blaah["Résultat"] = blaah.apply(resultat, axis=1)

In [25]:
blaah = pd.concat([blaah, pd.get_dummies(blaah["Résultat"])], axis=1)

### Nombre de buts dans les matches

In [26]:
blaah["Nb Buts"] = blaah["Buts_OM"] + blaah["Buts_Adversaire"]


In [27]:
alt.Chart(blaah[["Nb Buts"]]).mark_bar().encode(
    x=alt.X("Nb Buts:O", axis=alt.Axis(labelAngle=0)),
    y="count():Q",
    color=alt.Color("Nb Buts:O", scale=alt.Scale(scheme="greens")),
    tooltip=["Nb Buts:O", alt.Tooltip("count():Q", title="Nb Matches")]
).properties(
    width=DEFAULT_WIDTH
)

alt.Chart(...)

### Nombre de buts de l'OM

In [28]:
alt.Chart(blaah[["Buts_OM"]]).mark_bar().encode(
    x=alt.X("Buts_OM:O", title="Buts de l'OM", axis=alt.Axis(labelAngle=0)),
    y="count():Q",
    color="Buts_OM:O",
    tooltip=[alt.Tooltip("Buts_OM:O", title="Buts de l'OM"), alt.Tooltip("count():Q", title="Nb Matches")]
).properties(
    width=DEFAULT_WIDTH
)

alt.Chart(...)

### Nombre de buts de l'adversaire

In [29]:
alt.Chart(blaah[["Buts_Adversaire"]]).mark_bar().encode(
    x=alt.X("Buts_Adversaire:O", title="Buts de l'adversaire", axis=alt.Axis(labelAngle=0)),
    y="count():Q",
    color=alt.Color("Buts_Adversaire:O", scale=alt.Scale(scheme="reds")),
    tooltip=[alt.Tooltip("Buts_Adversaire:O", title="Buts de l'OM"), alt.Tooltip("count():Q", title="Nb Matches")]
).properties(
    width=DEFAULT_WIDTH
)

alt.Chart(...)

### Différence de But par Adversaire

In [30]:
by_adv = blaah.groupby("Adversaire").agg({"Publications": "sum"})

In [31]:
# Il va falloir traiter les doublons...

# * ASSE / Saint-Etienne / Saint-Étienne
# * TFC / Toulouse
# * MHSC / Montpellier
# * Gazélec / Gazéléc
# * Konyaspot / Konyasport
# * Salzbourg / Salzburg
# * Athletic / Bilbao

In [32]:
normalize = [
    {"dups": ["ASSE", "Saint-Etienne", "Saint-Étienne"], "norm": "Saint-Étienne"},
    {"dups": ["TFC", "Toulouse"], "norm": "Toulouse"},
    {"dups": ["MHSC", "Montpellier"], "norm": "Montpellier"},
    {"dups": ["Gazélec", "Gazéléc"], "norm": "Gazélec Ajaccio"},
    {"dups": ["Konyaspor", "Konyasport"], "norm": "Konyaspor"},
    {"dups": ["Salzbourg", "Salzburg"], "norm": "Salzburg"},
    {"dups": ["Athletic", "Bilbao"], "norm": "Athletic Bilbao"},
    {"dups": ["Atlético"], "norm": "Atletico Madrid"}
]

In [33]:
for dedup in normalize:
    blaah.loc[blaah["Adversaire"].isin(dedup["dups"]), "Adversaire"] = dedup["norm"]

In [34]:
by_adv = blaah.groupby("Adversaire").agg({"Publications": "sum", "Buts_OM": "sum", "Buts_Adversaire": "sum"})
by_adv.reset_index(inplace=True)

In [35]:
by_adv["Différence de Buts"] = by_adv["Buts_OM"] - by_adv["Buts_Adversaire"]

In [36]:
slider = alt.binding_range(min=by_adv["Publications"].min(), max=by_adv["Publications"].max(), step=1, name="Nombre Minimum de Matchs: ")
select_nb_matches = alt.selection_single(name="matches", fields=["Publications"], bind=slider, init={"Publications": 1})

alt.Chart(by_adv).mark_bar().encode(
    y="Différence de Buts",
    x=alt.X(
        "Adversaire:O", 
        sort=alt.EncodingSortField(field="Différence de Buts", order="ascending"),
        axis=alt.Axis(labelAngle=-45)
    ),
    color=alt.Color("Différence de Buts", scale=alt.Scale(scheme="redyellowgreen")),
    tooltip=["Adversaire", "Différence de Buts", alt.Tooltip("Publications", title="Nb Matches")]
).add_selection(
    select_nb_matches
).transform_filter(
    "datum.Publications >= matches_Publications"
). properties(height=600,width=1200)

alt.Chart(...)

### Différentiel Victoire / Défaite par Adversaire

In [37]:
by_adv = blaah.groupby("Adversaire").agg({"Publications": "sum", "Victoire": "sum", "Défaite": "sum", "Nul": "sum"})
by_adv.reset_index(inplace=True)
by_adv["Différence Victoire / Défaite"] = by_adv["Victoire"].astype(int) - by_adv["Défaite"].astype(int)

In [38]:
chart = alt.Chart(by_adv).mark_bar().encode(
    y="Différence Victoire / Défaite",
    x=alt.X(
        "Adversaire:O", 
        sort=alt.EncodingSortField(field="Différence Victoire / Défaite"),
        axis=alt.Axis(labelAngle=-45)
    ),
    color=alt.Color("Différence Victoire / Défaite", scale=alt.Scale(domain=[-10, 10], scheme="redyellowgreen")),
    tooltip=["Adversaire", "Victoire", "Nul", "Défaite", alt.Tooltip("Publications", title="Nb Matches")]
).add_selection(
    select_nb_matches
).transform_filter(
    "datum.Publications >= matches_Publications"
). properties(height=600,width=1200)

chart

alt.Chart(...)



___


# Les Verbes dans les Titres

In [39]:
blaah["Titre_Part2"] = blaah["Titre"].str.lower().str.extract(r"^.+(la\s+canebière\s+académiq?u?e[\w\s]+)")

In [40]:
blaah = blaah.dropna(subset=["Titre_Part2"])

In [41]:
blaah["Titre_Part2"] = blaah["Titre_Part2"].str.replace(r"la\s+canebière\s+académiq?u?e", "elle", regex=True)

In [42]:
fr = spacy.load('fr_core_news_sm')

In [43]:
blaah["Titre_Part2_doc"] = blaah["Titre_Part2"].apply(fr)

In [44]:
def extract_verb(x):
    v = [t.lemma_ for t in x if t.pos_ == "VERB"]
    if len(v) == 0:
        return np.nan
    return lemmatizer.lemmatize(v[0], "v")

blaah["Verbe"] = blaah["Titre_Part2_doc"].apply(extract_verb)

In [45]:
def group_verb(x: str) -> int:
    if x.endswith("er"):
        return 1

    if x.endswith("oir"):
        return 3

    if x.endswith("ir"):
        return 2

    return 3

verbes = pd.DataFrame(blaah["Verbe"].dropna().unique(), columns=["Verbe"])
verbes["groupe"] = verbes["Verbe"].apply(group_verb)

In [46]:
chart = alt.Chart(blaah[["Verbe", "Publications"]].dropna()).mark_bar().encode(
    x=alt.X(
        field="Verbe", 
        type="ordinal", 
        sort=alt.EncodingSortField(field="Occurences", order="descending"),
        axis=alt.Axis(labelAngle=-45, labelFontSize=14)
    ),
    y="Occurences:Q",
    color=alt.Color("Occurences:Q", scale=alt.Scale(scheme="greenblue")),
    tooltip=["Verbe", "Occurences:Q"]
).transform_aggregate(
    groupby=["Verbe"],
    Occurences="sum(Publications)"
).transform_filter(
    alt.datum.Occurences >= 1
)

chart.configure_header(
    titleColor='green',
    titleFontSize=14,
    labelColor='red',
    labelFontSize=14
)

chart

alt.Chart(...)



___


# Les Notes (Extraites du texte de l'académie)

In [47]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(blaah.iloc[0]["html"])

In [48]:
notes_re = re.compile(r"^(?P<joueur>[\w\s]+) \(.*,?\w*(?P<note>\d[\+\-]?)/5.*")
note_num = re.compile(r"^(?P<num>\d+)(?P<plus>[\+\-]?)$")

def extract_notes(row: pd.Series) -> pd.DataFrame:
    adversaire = row["Adversaire"]
    date = row["Date"]
    soup = BeautifulSoup(row["html"])
    data = []
    for st in soup.find_all(["strong", "b", "i", "em"]):
        m = notes_re.match(st.text)
        if m is None:
            continue
        
        m_num = note_num.match(m.group("note"))
        if m_num is None:
            continue
        
        plus_moins = m_num.group("plus")
        note_ = float(m_num.group("num")) 
        if len(plus_moins) > 0:
            note_ += 0.5 if plus_moins == "+" else -0.5
        data.append({"Adversaire": adversaire, "Date": date, "Joueur": m.group("joueur").replace("\n", " ").strip().title(), "Note (texte)": m.group("note"), "Note (nombre)": note_})
    return pd.DataFrame(data)



In [49]:
notes = pd.concat(blaah.apply(extract_notes, axis=1).values)

In [50]:
normalize = [
    {"dups": ["Amavi", "Amavier"], "norm": "Amavi"},
    {"dups": ["De Ceglie", "Détchéyé"], "norm": "De Ceglie"},
    {"dups": ["Leya Iseka", "Iseka Leya"], "norm": "Leya Iseka"},
    {"dups": ["Lopez", "Maxime Lopez"], "norm": "Lopez"},
    {"dups": ["Doria", "Diego Armando Maradoria"], "norm": "Doria"},
    {"dups": ["Njie", "Nvier"], "norm": "Njie"},
    {"dups": ["Batshuayi", "Michybre"], "norm": "Batshuayi"},
]

In [51]:
for dedup in normalize:
    notes.loc[notes["Joueur"].isin(dedup["dups"]), "Joueur"] = dedup["norm"]

In [52]:
stats = pd.DataFrame({"Académie": ["Académie", "Académie"], "Notes": ["Récupérées", "Non Récupérées"], "Nombre": [notes["Date"].nunique(), blaah.shape[0] - notes["Date"].nunique()]})

alt.Chart(stats).mark_bar().encode(
    x=alt.X("Académie:N", axis=alt.Axis(labels=False)),
    y=alt.Y("Nombre:Q", stack="zero"),
    color=alt.Color("Notes:N", legend=alt.Legend(title="Récupération Automatique"), scale=alt.Scale(domain=["Récupérées", "Non Récupérées"], range=["green", "red"])),
    tooltip="Nombre:Q"
)

alt.Chart(...)

In [53]:
by_joueur = notes.groupby("Joueur").agg({"Note (nombre)": ["count", "mean"]})
by_joueur.columns = by_joueur.columns.droplevel()
by_joueur.reset_index(inplace=True)

In [54]:
chart = alt.Chart(by_joueur).mark_bar().encode(
    x=alt.X(
        field="Joueur", 
        type="ordinal", 
        sort=alt.EncodingSortField(field="count", order="descending"),
        axis=alt.Axis(labelAngle=-45, labelFontSize=14)
    ),
    y=alt.Y("count:Q", title="Nombre de Notes"),
    color=alt.Color("count:Q", scale=alt.Scale(scheme="greenblue")),
    tooltip=["Joueur", alt.Tooltip("count:Q", title="# Notes")]
)

chart.configure_header(
    titleColor='green',
    titleFontSize=14,
    labelColor='red',
    labelFontSize=14
)

chart

alt.Chart(...)

In [55]:
slider = alt.binding_range(min=by_joueur["count"].min(), max=by_joueur["count"].max(), step=1, name="Nombre Minimum de Notes: ")
select_nb_matches = alt.selection_single(name="matches", fields=["count"], bind=slider, init={"count": 1})

chart = alt.Chart(by_joueur).mark_bar().encode(
    x=alt.X(
        field="Joueur", 
        type="ordinal", 
        sort=alt.EncodingSortField(field="mean", order="descending"),
        axis=alt.Axis(labelAngle=-45, labelFontSize=14)
    ),
    y=alt.Y("mean:Q", title="Moyenne"),
    color=alt.Color("mean:Q", scale=alt.Scale(scheme="redyellowgreen", domain=[0.0, 5.0])),
    tooltip=["Joueur", alt.Tooltip("count:Q", title="# Notes"), alt.Tooltip("mean:Q", title="Moyenne")]
).add_selection(
    select_nb_matches
).transform_filter(
    'datum.count >= matches_count'
)

chart.configure_header(
    titleColor='green',
    titleFontSize=14,
    labelColor='red',
    labelFontSize=14
)

chart

alt.Chart(...)

In [56]:
nb_notes = notes["Joueur"].value_counts()

alt.Chart(notes[notes["Joueur"].isin(nb_notes[nb_notes >= 10].index)][["Joueur", "Note (nombre)"]]).transform_density(
    'Note (nombre)',
    as_=['Note', 'density'],
    extent=[0, 5],
    groupby=['Joueur']
).mark_area(orient='horizontal').encode(
    y='Note:Q',
    color='Joueur:N',
    x=alt.X(
        'density:Q',
        stack='center',
        impute=None,
        title=None,
        axis=alt.Axis(labels=False, values=[0],grid=False, ticks=True),
    ),
    column=alt.Column(
        'Joueur:N',
        header=alt.Header(
            titleOrient='bottom',
            labelOrient='bottom',
            labelPadding=0,
            labelFontWeight="bold",
            labelBaseline="bottom",
            labelAngle=-45
        ),
    )
).properties(
    width=50
).configure_facet(
    spacing=0
).configure_view(
    stroke=None
)

alt.Chart(...)

# Toutes les Notes, Tous les Joueurs, Tous les Matches

In [57]:
by_joueur_by_note = notes.groupby(["Joueur", "Note (nombre)"]).agg({"Date": "count"})
by_joueur_by_note = by_joueur_by_note.reset_index()
by_joueur_by_note.columns = ["Joueur", "Note", "count"]

In [58]:
alt.Chart(by_joueur_by_note, width=1200).mark_bar().encode(
    x=alt.X(
        'Joueur:N',
        title=None,
        axis=alt.Axis(labelAngle=-45),
        scale=alt.Scale(),
    ),
    y=alt.Y(
        'Note:Q', 
        stack="normalize", 
        axis=alt.Axis(labels=False, ticks=False),
        title=""
    ),
    order=alt.Order("Note:Q", sort="ascending"),
    color=alt.Color('Note:Q', scale=alt.Scale(scheme="redyellowgreen", domain=[0.0, 5.0])),
    tooltip=["Joueur", "Note", alt.Tooltip("count", title="Nombre de fois")]
)

alt.Chart(...)

In [59]:
notes["Match"] = notes["Adversaire"] + " " + notes["Date"].dt.strftime("%d/%m/%Y")
notes["sort"] = notes["Date"].dt.strftime("%Y%m%d")
notes["count"] = 1

In [60]:
alt.Chart(notes).mark_point(shape="square", size=2).encode(
    x=alt.X(
        "Match:O",
        sort=alt.EncodingSortField("sort", order="ascending"), 
        axis=alt.Axis(labels=False, ticks=False)
        ),
    y=alt.Y(
        "Joueur:N",
        axis=alt.Axis(labels=False, ticks=False)
        ),
    color=alt.Color("Note (nombre):Q", scale=alt.Scale(scheme="redyellowgreen")),
    #size=alt.Size("count:Q", ),
    tooltip=["Joueur", "Note (nombre)", "Adversaire", "Date"]
).properties(
    height=400,
    width=1200
)

alt.Chart(...)

In [61]:
width = 1200
nb_matches = notes["Match"].nunique()
nb_joueurs = notes["Joueur"].nunique()

select_joueurs = alt.selection_multi(fields=["Joueur"])

joueurs = alt.Chart(pd.DataFrame({"Joueur": notes["Joueur"].unique()})).mark_rect().encode(
    x=alt.X(
        "Joueur:N",
        axis=alt.Axis(labelAngle=-45),
    ),
    color=alt.condition(select_joueurs, alt.Color('Joueur:N', legend=None), alt.value('lightgray'))
).add_selection(
    select_joueurs
).properties(
    width=width
)

heatmap = alt.Chart(notes).mark_point(shape="square", size=2).encode(
    x=alt.X(
        "Match:O",
        sort=alt.EncodingSortField("sort", order="ascending"), 
        axis=alt.Axis(labels=False, ticks=False)
        ),
    y=alt.Y(
        "Joueur:N",
        axis=alt.Axis(labels=False, ticks=False)
        ),
    color=alt.Color("Note (nombre):Q", scale=alt.Scale(scheme="redyellowgreen")),
    #size=alt.Size("count:Q", ),
    tooltip=["Joueur", "Note (nombre)", "Adversaire", "Date"]
).properties(
    height=400,
    width=width
).transform_filter(
    select_joueurs
)

joueurs & heatmap

alt.VConcatChart(...)

# Les notes de l'OM par match

In [62]:
by_match_by_note = notes.groupby(["Match", "Note (nombre)"]).agg({"Joueur": "count", "sort": "max"}).reset_index()

In [63]:
alt.Chart(by_match_by_note).mark_circle().encode(
    y=alt.Y(
        'Match:N',
        title=None,
        axis=alt.Axis(ticks=False, grid=False, labelPadding=20),
        sort=alt.EncodingSortField("sort", op="max")
    ),
    x=alt.X(
        'Note (nombre):Q',
        axis=alt.Axis(grid=False)
    ),
    color=alt.Color(
        'Note (nombre):Q', 
        legend=None, 
        scale=alt.Scale(domain=[0.0,5.0], scheme="redyellowgreen")
    ),
    size=alt.Size(
        "Joueur:Q",
        legend=None,
        scale=alt.Scale(domain=[0.0, by_match_by_note["Joueur"].max()], range=[0.0, 2000.0])
    ),
    tooltip=["Match:N", alt.Tooltip("Note (nombre):Q", title="Note"), alt.Tooltip("Joueur:N", title="# Joueurs")]
)

alt.Chart(...)

In [64]:
alt.Chart(by_match_by_note).mark_bar().encode(
    y=alt.Y(
        'Match:N',
        title=None,
        axis=alt.Axis(ticks=False, grid=False, labelPadding=20),
        sort=alt.EncodingSortField("sort", op="max")
    ),
    x=alt.X(
        'Note (nombre):Q',
        stack="normalize",
        axis=alt.Axis(grid=False)
    ),
    color=alt.Color(
        'Note (nombre):Q', 
        legend=None, 
        scale=alt.Scale(domain=[0.0, 5.0], scheme="redyellowgreen")
    ),
    tooltip=["Match:N", alt.Tooltip("Note (nombre):Q", title="Note"), alt.Tooltip("Joueur:N", title="# Joueurs")]
)

alt.Chart(...)

In [65]:
notes["sort_int"] = notes["sort"].astype(int)

In [66]:
alt.Chart(notes[["Match", "sort_int", "Note (nombre)"]].rename(columns={"Note (nombre)": "Note"}).sort_values("sort_int")).mark_boxplot(extent="min-max").encode(
    x=alt.X(
        'Match:N',
        title=None,
        axis=alt.Axis(ticks=False, grid=False, labelAngle=-45),
        sort=alt.EncodingSortField(order=None)
    ),
    y=alt.Y(
        'Note:Q',
        axis=alt.Axis(grid=False,)
    ),
    color=alt.Color("mean(Note):Q", scale=alt.Scale(domain=[0.0, 5.0], scheme="redyellowgreen")),
    tooltip=["Match:N"]
)

alt.Chart(...)